In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
#pos = "rb"

In [45]:
def get_per_game(df):
    cols = df.columns
    
    for c in cols:
        if "RUSHING" in c or "RECEIVING" in c or "FL" in c or "PASSING" in c:
            df[c+"/G"] = df[c]/df["G_MISC"]
            #new_names.append()
    
    return df

In [4]:
def find_last_ewm(x):
    #print(x.columns)
    #print(x.name)
    if x.dtype != "O" and x.name!="pid":
        return x.ewm(halflife=2).mean().values[-1]
        
            
    #elif x.dtype == "O":
     #   values.append(" ")
    #return values
    #print(x.ewm(alpha=0.5))

In [73]:
def find_comp_df(pos, season_int, how="past"):
    if how =="ns":
        past_data = pd.read_csv(f"raw data/{pos}_raw.csv")
        ns_data = pd.read_csv(f"ns data/{pos}_ns.csv")
        std_dev = ns_data[["Yahoo", "Sleeper", "RTSports"]].std(axis=1)
        
        past_data = past_data[past_data.Season != past_data.Season.values[-1]]
        ns_data = ns_data.rename({"Rank":"Rank_y"}, axis=1)
        line = pd.concat([past_data, ns_data])#.append(ns_data)
        line = line.drop("Player", axis=1)
        line = line.rename({"Player Team (Bye)": "Player"}, axis=1)
        
        
    else:
        line = pd.read_csv(f"raw data/{pos}_raw.csv")
    #return line
    if pos!= "qb":
        base_line = ["Player", "pid", "Season", "ATT_RUSHING", "YDS_RUSHING", "TD_RUSHING", "REC_RECEIVING", "TGT_RECEIVING", "YDS_RECEIVING", "TD_RECEIVING", "FL_MISC", "AVG", "G_MISC", "FPTS/G_MISC", "FPTS_MISC"]
    else:
        base_line = ["Player", "pid", "Season", 'CMP_PASSING', 'ATT_PASSING', 'PCT_PASSING','YDS_PASSING', 'Y/A_PASSING', 'TD_PASSING', 'INT_PASSING','SACKS_PASSING', 'ATT_RUSHING', 'YDS_RUSHING', 'TD_RUSHING', "FL_MISC", "AVG", "G_MISC", "FPTS/G_MISC", "FPTS_MISC"]

    gs = line[base_line]
    #return gs
    pg_df = get_per_game(gs.copy())

    if pos != "qb":
        classes = ["_RUSHING", "_RECEIVING"]
        for c in classes:

            if c == "_RUSHING":
                pg_df["YDS/ATT"+c] =pg_df["YDS"+c] /pg_df["ATT"+c]
                pg_df["TD/ATT"+c] =pg_df["TD"+c] /pg_df["ATT"+c]
            else:
                pg_df["YDS/REC"+c] =pg_df["YDS"+c] /pg_df["REC"+c]
                pg_df["YDS/TGT"+c] =pg_df["YDS"+c] /pg_df["TGT"+c]
                pg_df["REC/TGT"+c] =pg_df["REC"+c] /pg_df["TGT"+c]
                pg_df["TD/TGT"+c] =pg_df["TD"+c] /pg_df["TGT"+c]

    agg_feats_base = pg_df.drop("AVG", axis=1).columns[3:].values

    agg_feats = []
    for f in agg_feats_base:
        if "/" in f:
            agg_feats.append(f)
    
    #return agg_feats
    adp = "AVG"
    if how == "ns":
        seasons = np.arange(2018, season_int+1)
    else:
        seasons = np.arange(2018, season_int)

    full_bdf = pd.DataFrame()
    full_bdfs = []
    for s in tqdm(seasons):
        bdf = pg_df[pg_df.Season == s][["pid", "Player", "Season","AVG", "FPTS/G_MISC"]]
        bdf = bdf.rename({"FPTS/G_MISC": "FPTS_TG"}, axis=1)
        #print(len(bdf))
        bdf =  bdf.merge(pg_df[(pg_df.Season < s) & (pg_df.Season > s-4)].groupby("pid")[agg_feats].mean().reset_index(), how="left", on="pid")

        #return pg_df[(pg_df.Season < s) & (pg_df.Season > s-5)]
        ewm_df = pg_df[(pg_df.Season < s) & (pg_df.Season > s-4)].reset_index(drop=True).groupby("pid").agg(find_last_ewm)[agg_feats]

        bdf =  bdf.merge(ewm_df.reset_index(), how="left", on="pid", suffixes=["_avg", "_ewm"])

        #ls = pg_df[(pg_df.Season == s-1)].groupby("pid").mean()[agg_feats]
        #ls_cols = [a+"_ls" for a in ls.columns]        #ls.columns = ls_cols

        #bdf = bdf.merge(ls.reset_index(), how="left", on="pid")


        #full_bdf = full_bdf.append(bdf)
        full_bdfs.append(bdf)

    full_bdf = pd.concat(full_bdfs)
    drafted = full_bdf[~full_bdf.AVG.isna()]

    is_rookie = []
    career_years = []
    
    for i, s in zip(drafted.pid.values, drafted.Season.values):
        last_apps = pg_df[(pg_df.Season < s) & (pg_df.pid == i)]
        if len(last_apps) > 0:
            is_rookie.append(0)
        else:
            is_rookie.append(1)


        career_years.append(len(last_apps))
    
    drafted = drafted.copy()
    drafted.loc[:, "Rookie"] = is_rookie
    drafted.loc[:, "Career_Years"] = career_years
    
    if how == "ns":
        drafted = drafted[drafted.Season == season_int]
        std_dev = std_dev.fillna(std_dev.dropna().tail(5).mean())
        drafted["STD"] = std_dev
    return drafted

In [19]:
pos_data = find_comp_df('rb', 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.04it/s]


In [85]:
pos_data.to_csv(f"processed data/{pos}_proc_data.csv", index=False)

In [20]:
wr_data = find_comp_df('wr', 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.25s/it]


In [87]:
wr_data.to_csv(f"processed data/wr_proc_data.csv", index=False)

In [21]:
te_data = find_comp_df('te', 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.38it/s]


In [11]:
te_data.to_csv(f"processed data/te_proc_data.csv", index=False)

In [31]:
a = pd.read_csv("raw data/qb_raw.csv")

In [49]:
qb_data = find_comp_df("qb", 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.32it/s]


In [51]:
qb_data.to_csv(f"processed data/qb_proc_data.csv", index=False)

In [76]:
ns_proc = find_comp_df('rb', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.10it/s]


In [82]:
ns_proc.to_csv("processed data/rb_ns_proc.csv", index=False)

In [83]:
wr_proc = find_comp_df('wr', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.34s/it]


In [84]:
wr_proc.to_csv("processed data/wr_ns_proc.csv", index=False)

In [85]:
te_proc =  find_comp_df('te', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.23it/s]


In [86]:
te_proc.to_csv("processed data/te_ns_proc.csv", index=False)

In [87]:
qb_proc =  find_comp_df('qb', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.26it/s]


In [88]:
qb_proc.to_csv("processed data/qb_ns_proc.csv", index=False)

In [89]:
wr_proc

,pid,Player,Season,AVG,FPTS_TG,FPTS/G_MISC_avg,ATT_RUSHING/G_avg,YDS_RUSHING/G_avg,TD_RUSHING/G_avg,REC_RECEIVING/G_avg,...,FL_MISC/G_ewm,YDS/ATT_RUSHING_ewm,TD/ATT_RUSHING_ewm,YDS/REC_RECEIVING_ewm,YDS/TGT_RECEIVING_ewm,REC/TGT_RECEIVING_ewm,TD/TGT_RECEIVING_ewm,Rookie,Career_Years,STD
0,19236,Justin Jefferson MIN (13),2023,1.3,NaN,16.200000,0.216912,0.786765,0.019608,6.460784,...,0.018846,3.919120,0.113270,14.801174,10.091920,0.681862,0.051570,0,3,0.577350
1,19788,Ja'Marr Chase CIN (7),2023,3.0,NaN,15.400000,0.398190,0.925339,0.000000,5.728507,...,0.114486,2.179899,0.000000,14.483406,9.281075,0.642444,0.081412,0,2,0.000000
2,16433,Cooper Kupp LAR (10),2023,5.3,NaN,16.866667,0.500654,3.012200,0.037037,7.665359,...,0.065445,5.928466,0.050342,11.605638,8.799384,0.758043,0.060058,0,6,0.577350
3,15802,Tyreek Hill MIA (10),2023,7.3,NaN,16.600000,0.602614,5.243137,0.064052,6.443137,...,0.018846,7.632014,0.099578,13.409375,9.195226,0.686810,0.061962,0,7,0.577350
4,13981,Stefon Diggs BUF (13),2023,11.3,NaN,15.366667,0.040441,-0.037990,0.000000,6.822304,...,0.000000,-1.666667,0.000000,12.541462,8.713742,0.694011,0.062401,0,8,1.154701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,25287,Cedric Tillman CLE (5),2023,301.0,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,22.627417
104,18585,Greg Dortch ARI (14),2023,286.0,NaN,3.250000,0.318750,3.775000,0.000000,1.925000,...,0.036612,13.623212,0.000000,7.331881,6.345478,0.890165,0.018306,0,3,21.496046
105,22989,Kayshon Boutte NE (11),2023,293.0,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,21.496046
106,19344,Denzel Mims DET (9),2023,313.0,NaN,3.300000,0.000000,0.000000,0.000000,1.525926,...,0.000000,NaN,NaN,16.503783,7.061616,0.429211,0.000000,0,3,21.496046


In [58]:
ns_data = pd.read_csv(f"ns data/qb_ns.csv")

In [71]:
z = ns_data[["Yahoo", "Sleeper", "RTSports"]].std(axis=1)

0      1.000000
1      4.358899
2      1.154701
3      3.055050
4      3.605551
5      5.000000
6      2.000000
7      3.000000
8      9.451631
9      4.509250
10     4.041452
11     6.000000
12     4.358899
13    12.124356
14     7.000000
15    15.011107
16    19.502137
17    28.378396
18    16.196707
19    23.115651
20    25.006666
21    24.440404
22    10.263203
23    34.117444
24    27.934447
25    32.868425
26    51.081634
27    56.107041
28     7.371115
29    26.162951
30    24.748737
31    20.506097
32    43.840620
33    38.183766
34    28.991378
35    31.254120
36    31.254120
37    31.254120
38    31.254120
39    31.254120
40    31.254120
dtype: float64